In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install --upgrade tensorflow==1.14.0

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
# 코랩에서 사용시 ! 붙이기
#!git clone https://github.com/tensorflow/models/

In [ ]:
cd models/research/

In [ ]:
!pip install Cython
!apt-get install protobuf-compiler python-pil python-lxml python-tk

In [ ]:
import os
import pathlib

In [ ]:
cd /content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter06/models/research/

In [ ]:
# 코랩에서 사용시 ! 붙이기
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%set_env PYTHONPATH=/content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter06/models/research/:/content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter06/models/research/slim

In [ ]:
pip install tf_slim

In [ ]:
!python object_detection/builders/model_builder_test.py

In [ ]:
print(os.getcwd())

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
cd  /content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter06/models/research/object_detection/utils/

In [ ]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(3, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_NAME = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
print(PATH_TO_FROZEN_GRAPH)

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)